# SpeakUp2.0: Splitting Multiple Videos



## Info documents
location Repositor :https://github.com/DavAhm/Mobile-Multimodal-Lab/tree/main/2_PREPROCESSING/2_MOTION_TRACKIN

location Jupyter Noteboo  
##  Requirements
Please install the necessary packages in requirements.
## OVERVIEWE
This Python script splits single videos containing multiple cameras views into individual videos of each separate view using OpenCV. 
It outputs the split video files into a specific folder structure, making them ready for further processing, such as motion tracking with tools like OpenPose or Pose2Sim. Below is an overview of the key steps in the code:

0. Importing Necessary Packages:
             see requirements.txt for all necessary packages

1. Defining Relevant Directories, Variables, and Functions:
         Working with relative paths, the scripts defines input and output folders.
         Video extension (e.g., *.avi) and corresponding codec extension (e.g., XVID) can be ajusted by the user accordingly.
         The current script assumes 3 cameras for each vidoe, each of them with an euqal height and width.
         The split_camera_views function is defined to handle the core task of splitting a video file into separate camera views. This function processes each video file, extracting frames and dividing them into distinct sections corresponding to different camera views. The split videos are then saved to specified output files.


2. and 2a. In **2** the script automatically scans a specified directory to identify and collect all video files with a specific extension (e.g., .avi). This allows for batch processing of multiple videos without requiring manual file selection.
   In **2a**, the the script provides an option for the user to manually select one or more video files through a graphical file dialog. This feature offers flexibility for the user to choose specific videos from any directory for processing.

3. The scripts loops through the selected or identified video files and splitting each one into separate camera views:
For each video file, the script extracts the filename and generates output file paths for each camera view (e.g., _cam1.avi, _cam2.avi, _cam3.avi).
The split_camera_views function is called to process the video, splitting it into three separate camera views and saving them in a structured folder, such as a raw-2d subfolder within the output directory.   

## 0. Importing Necessary Packages


In [1]:
import os                                 # Importing the os module which provides functions for interacting with the operating system
from moviepy.editor import (              # Importing functions from the MoviePy library, which is used for video editing.
                VideoFileClip,                # VideoFileClip allows you to load a video file and manipulate it as a clip object.
                concatenate_videoclips)       # concatenate_videoclips allows you to concatenate multiple video clips into one video.
import cv2                                # Importing cv2 module from OpenCV library, used for computer vision tasks (provides various functions to process images and videos).
import subprocess                         # Importing subprocess, commonly used for running external commands or scripts from within a Python script.
import tkinter                            # Importning tkinter, whch crates graphical user interfaces (GUIs) and provides a variety of widgets and controls to build desktop applications.
from tkinter import filedialog                # importing filedialog from tkinter, is a module in tkinter, which allows the user to select files or directories through a dialog window.
from tqdm import tqdm                     # Importing tqdm for the progress bar
import re

print("Everything was imported succesfully") #as terminal

Everything was imported succesfully


## 1. Defining the Relevant Directories, Variables & Functions

In [4]:
# ------------ DIRECTORIES -----------------------------
input_folder  = r'\\fileserver.dccn.nl\project\3025011.01\Davide_MultimodalLab\3_DATA_ANALYSIS\1_MOTION_TRACKING\2_Video_Calibration\calibration_charuco_raw'    # Relative path to the input folder with the raw video files
output_folder = r'\\fileserver.dccn.nl\project\3025011.01\Davide_MultimodalLab\3_DATA_ANALYSIS\1_MOTION_TRACKING\2_Video_Calibration\calibration_charuco_split'  # output folder where the split videos will be saved (relative path) 

print("Input folder =", os.path.abspath(input_folder))
print("Output folder =", os.path.abspath(output_folder))


# ----------- VARIABLES --------------------------
input_video_extension = '.avi'  # Video format extension avi. Change as needed (e.g., .mp4)
codec_extension  = 'mp4v'  # Codec for avi. Consider other codecs for different extensions:
                            # '.mp4': 'libx264',
                            # '.avi': 'libxvid',
                            # '.mov': 'libx264',
                            # '.mkv': 'libx264',
                            # '.flv': 'flv',
                            
output_video_extension = '.mp4'  # Output video format extension. Change as needed (e.g., .avi)

num_cameras = 3          # Number of cameras for each "raw video". Change accordingly. 

condition_pattern_matching = re.compile(r'.(Vision|NoVision)_(Movement|NoMovement)')

keyword = 'charuco'  # Keyword to search for in the file name. Change as needed.




# ----------- FUNCTIONS -------------------------
def split_camera_views(input_file, output_files, num_cameras=3, codec_extension='mp4v'):
    """
    This split_camera_views function takes an input video file and splits it into three separate video files, 
    each corresponding to a different camera view. It assumes that the input video is composed 
    of three side-by-side camera views. The function outputs three video files, each containing 
    one of the camera views.

    Parameters:
    input_file (str): The path to the input video file.
    output_files (list of str): A list containing the paths to the output video files for each camera view.
    num_cameras (int): The number of camera views in the input video. Default is 3.
    codec_extension (str): The codec extension to be used for the output videos. Default is 'mp4v'.
    """
    
    cap = cv2.VideoCapture(input_file)  # Open the video file

    # Extract relevant information from the video
    width_per_camera = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) // num_cameras  # Calculate width per camera
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  # Get the height of the video (same for all cameras)
    frame_rate = int(cap.get(cv2.CAP_PROP_FPS))  # Get the frame rate of the video (same for all cameras)

    # Create VideoWriters for each camera
    fourcc = cv2.VideoWriter_fourcc(*codec_extension)  # Define the codec for the output videos
    out_cam1 = cv2.VideoWriter(output_files[0], fourcc, frame_rate, (width_per_camera, height))  # Create VideoWriter for the first camera
    out_cam2 = cv2.VideoWriter(output_files[1], fourcc, frame_rate, (width_per_camera, height))  # Create VideoWriter for the second camera
    out_cam3 = cv2.VideoWriter(output_files[2], fourcc, frame_rate, (width_per_camera, height))  # Create VideoWriter for the third camera

    while True:  # Loop to read frames from the video
        ret, frame = cap.read()  # Read a frame from the video

        # Check if the frame is None (end of video)
        if frame is None:  # If no frame is returned, break the loop
            break

        # Break the frame into three parts
        camera1_frame = frame[:, :width_per_camera, :]                    # Extract the first camera view
        camera2_frame = frame[:, width_per_camera:2*width_per_camera, :]  # Extract the second camera view
        camera3_frame = frame[:, 2*width_per_camera:, :]                  # Extract the third camera view

        # Display each camera view separately (optional)
        cv2.imshow('Camera 1', camera1_frame)  # Display the first camera view
        # cv2.imshow('Camera 2', camera2_frame)  # Display the second camera view
        # cv2.imshow('Camera 3', camera3_frame)  # Display the third camera view

        # Write frames to video files
        out_cam1.write(camera1_frame)  # Write the first camera view to its file
        out_cam2.write(camera2_frame)  # Write the second camera view to its file
        out_cam3.write(camera3_frame)  # Write the third camera view to its file

        if cv2.waitKey(1) == 27:  # Break the loop if 'Esc' key is pressed
            break

    # Release VideoWriters and VideoCapture
    out_cam1.release()  # Release the VideoWriter for the first camera
    out_cam2.release()  # Release the VideoWriter for the second camera
    out_cam3.release()  # Release the VideoWriter for the third camera
    cap.release()  # Release the VideoCapture
    cv2.destroyAllWindows()  # Close all OpenCV windows

print('Function split_camera_views created successfully')

Input folder = \\fileserver.dccn.nl\project\3025011.01\Davide_MultimodalLab\3_DATA_ANALYSIS\1_MOTION_TRACKING\2_Video_Calibration\calibration_charuco_raw
Output folder = \\fileserver.dccn.nl\project\3025011.01\Davide_MultimodalLab\3_DATA_ANALYSIS\1_MOTION_TRACKING\2_Video_Calibration\calibration_charuco_split
Function split_camera_views created successfully


## 2. Identifying Relevant Videos in the input_folder 

In [6]:
video_files = [ ]    # Initialize an empty list to store paths of video files

# Traverse through the directory and its subdirectories to find XDF files
try: 
    for root, dirs, videos in os.walk(os.path.abspath(input_folder)):  # 1st loop iterating over the results returned by os.walk()
        
        for video in videos:                               # 2nd loop iterating through each file in the current directory
            
            if  keyword in video and video.endswith(input_video_extension):            # checking if the video has the specified video_extension (e.g., '.avi') 
                
                video_files.append(os.path.join(root, video))   # if the video is an .avi file, append its full path to the  list
    
    if video_files:           
        print('We have idenified the following vides: ' + str(video_files))

    else:
        print('No video files found in the specified directory')

except Exception as e:
    print('An error occurred:', e)

We have idenified the following vides: ['\\\\fileserver.dccn.nl\\project\\3025011.01\\Davide_MultimodalLab\\3_DATA_ANALYSIS\\1_MOTION_TRACKING\\2_Video_Calibration\\calibration_charuco_raw\\pilot_01_d2_calibration_charuco.avi', '\\\\fileserver.dccn.nl\\project\\3025011.01\\Davide_MultimodalLab\\3_DATA_ANALYSIS\\1_MOTION_TRACKING\\2_Video_Calibration\\calibration_charuco_raw\\pilot_01_d3_calibration_charuco.avi', '\\\\fileserver.dccn.nl\\project\\3025011.01\\Davide_MultimodalLab\\3_DATA_ANALYSIS\\1_MOTION_TRACKING\\2_Video_Calibration\\calibration_charuco_raw\\pilot_02_d2_calibration_charuco.avi', '\\\\fileserver.dccn.nl\\project\\3025011.01\\Davide_MultimodalLab\\3_DATA_ANALYSIS\\1_MOTION_TRACKING\\2_Video_Calibration\\calibration_charuco_raw\\pilot_02_d3_calibration_charuco.avi', '\\\\fileserver.dccn.nl\\project\\3025011.01\\Davide_MultimodalLab\\3_DATA_ANALYSIS\\1_MOTION_TRACKING\\2_Video_Calibration\\calibration_charuco_raw\\pilot_04_d2_calibration_charuco.avi', '\\\\fileserver.dccn

## 2a. Alternatively, the user can select their own Videos from any directory. 

In [6]:
root = tkinter.Tk()
root.attributes('-topmost',True)
root.iconify()

video_files = filedialog.askopenfilename(title="Select one or multiple Videos in the format" + str(input_video_extension), filetypes=[("Video Files", str(input_video_extension))], multiple = 'True')

root.destroy()

# Convert the tuple returned by askopenfilenames() to a list
video_files = list(video_files)

print('You have selected the following Videos: ' + str(video_files))

You have selected the following Videos: ['C:/Users/ahmar/OneDrive - Radboud Universiteit/SpeakUp_temp/pilot_06_d2_calibration_charuco_output_compr - Trim.avi']


## 3. Splitting Videos (Loop)

In [7]:
### FOR CALIBRATION VIDEOS

# loop over files in folder and split them
for file in tqdm(video_files, desc="Splitting the videos: ", unit="file"):

    # Get the name of the file without the extension
    filename = os.path.splitext(os.path.basename(file))[0]
    
    subfolder_path = os.path.join(output_folder, filename)

    
    # Check if the subfolder exists, if not create it
    if not os.path.exists(subfolder_path):
        os.makedirs(subfolder_path)

    # create output file names for each camera, and save the three videos into ouput_folder with their correspodning camera angle 
    output_files = [
        os.path.join(subfolder_path, filename + '_cam1' + output_video_extension),
        os.path.join(subfolder_path, filename + '_cam2' + output_video_extension),
        os.path.join(subfolder_path, filename + '_cam3' + output_video_extension)
    ]

    # Make sure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    print("saving Split Videos as: " + str(output_files))


    # Splitting the videos using the function split_camera view
    split_camera_views(file, output_files)

print('Done, you can now look into the folder: ' + output_folder)

Splitting the videos:   0%|          | 0/7 [00:00<?, ?file/s]

saving Split Videos as: ['\\\\fileserver.dccn.nl\\project\\3025011.01\\Davide_MultimodalLab\\3_DATA_ANALYSIS\\1_MOTION_TRACKING\\2_Video_Calibration\\calibration_charuco_split\\pilot_01_d2_calibration_charuco\\pilot_01_d2_calibration_charuco_cam1.mp4', '\\\\fileserver.dccn.nl\\project\\3025011.01\\Davide_MultimodalLab\\3_DATA_ANALYSIS\\1_MOTION_TRACKING\\2_Video_Calibration\\calibration_charuco_split\\pilot_01_d2_calibration_charuco\\pilot_01_d2_calibration_charuco_cam2.mp4', '\\\\fileserver.dccn.nl\\project\\3025011.01\\Davide_MultimodalLab\\3_DATA_ANALYSIS\\1_MOTION_TRACKING\\2_Video_Calibration\\calibration_charuco_split\\pilot_01_d2_calibration_charuco\\pilot_01_d2_calibration_charuco_cam3.mp4']


Splitting the videos:  14%|█▍        | 1/7 [00:52<05:17, 52.92s/file]

saving Split Videos as: ['\\\\fileserver.dccn.nl\\project\\3025011.01\\Davide_MultimodalLab\\3_DATA_ANALYSIS\\1_MOTION_TRACKING\\2_Video_Calibration\\calibration_charuco_split\\pilot_01_d3_calibration_charuco\\pilot_01_d3_calibration_charuco_cam1.mp4', '\\\\fileserver.dccn.nl\\project\\3025011.01\\Davide_MultimodalLab\\3_DATA_ANALYSIS\\1_MOTION_TRACKING\\2_Video_Calibration\\calibration_charuco_split\\pilot_01_d3_calibration_charuco\\pilot_01_d3_calibration_charuco_cam2.mp4', '\\\\fileserver.dccn.nl\\project\\3025011.01\\Davide_MultimodalLab\\3_DATA_ANALYSIS\\1_MOTION_TRACKING\\2_Video_Calibration\\calibration_charuco_split\\pilot_01_d3_calibration_charuco\\pilot_01_d3_calibration_charuco_cam3.mp4']


Splitting the videos:  29%|██▊       | 2/7 [01:14<02:51, 34.33s/file]

...

saving Split Videos as: ['\\\\fileserver.dccn.nl\\project\\3025011.01\\Davide_MultimodalLab\\3_DATA_ANALYSIS\\1_MOTION_TRACKING\\2_Video_Calibration\\calibration_charuco_split\\pilot_05_d2_calibration_charuco\\pilot_05_d2_calibration_charuco_cam1.mp4', '\\\\fileserver.dccn.nl\\project\\3025011.01\\Davide_MultimodalLab\\3_DATA_ANALYSIS\\1_MOTION_TRACKING\\2_Video_Calibration\\calibration_charuco_split\\pilot_05_d2_calibration_charuco\\pilot_05_d2_calibration_charuco_cam2.mp4', '\\\\fileserver.dccn.nl\\project\\3025011.01\\Davide_MultimodalLab\\3_DATA_ANALYSIS\\1_MOTION_TRACKING\\2_Video_Calibration\\calibration_charuco_split\\pilot_05_d2_calibration_charuco\\pilot_05_d2_calibration_charuco_cam3.mp4']


Splitting the videos:  86%|████████▌ | 6/7 [04:03<00:44, 44.44s/file]

saving Split Videos as: ['\\\\fileserver.dccn.nl\\project\\3025011.01\\Davide_MultimodalLab\\3_DATA_ANALYSIS\\1_MOTION_TRACKING\\2_Video_Calibration\\calibration_charuco_split\\pilot_06_d2_calibration_charuco\\pilot_06_d2_calibration_charuco_cam1.mp4', '\\\\fileserver.dccn.nl\\project\\3025011.01\\Davide_MultimodalLab\\3_DATA_ANALYSIS\\1_MOTION_TRACKING\\2_Video_Calibration\\calibration_charuco_split\\pilot_06_d2_calibration_charuco\\pilot_06_d2_calibration_charuco_cam2.mp4', '\\\\fileserver.dccn.nl\\project\\3025011.01\\Davide_MultimodalLab\\3_DATA_ANALYSIS\\1_MOTION_TRACKING\\2_Video_Calibration\\calibration_charuco_split\\pilot_06_d2_calibration_charuco\\pilot_06_d2_calibration_charuco_cam3.mp4']


Splitting the videos: 100%|██████████| 7/7 [05:07<00:00, 43.95s/file]

Done, you can now look into the folder: \\fileserver.dccn.nl\project\3025011.01\Davide_MultimodalLab\3_DATA_ANALYSIS\1_MOTION_TRACKING\2_Video_Calibration\calibration_charuco_split


In [9]:
### FOR SPEECH VIDEOS

# loop over files in folder and split them
for file in tqdm(video_files, desc="Splitting the videos: ", unit="file"):

    # Get the name of the file without the extension
    filename = os.path.splitext(os.path.basename(file))[0]
    
    subfolder_path = os.path.join(output_folder, filename)
    
    # Check if the subfolder exists, if not create it
    if not os.path.exists(subfolder_path):
        os.makedirs(subfolder_path)

    # create output file names for each camera, and save the three videos into ouput_folder with their correspodning camera angle 
    output_files = [
        os.path.join(subfolder_path, f'{filename}_cam1{output_video_extension}'),
        os.path.join(subfolder_path, f'{filename}_cam2{output_video_extension}'),
        os.path.join(subfolder_path, f'{filename}_cam3{output_video_extension}')
    ]

    print("saving Split Videos as: " + str(output_files))


    # Splitting the videos using the function split_camera view
    split_camera_views(file, output_files)

print('Done, you can now look into the folder: ' + output_folder)

Splitting the videos:   0%|          | 0/40 [00:00<?, ?file/s]

Participant: P1
Condition: NoVision_Movement
Trial: trial_0
saving Split Videos as: ['./video_split/P1\\NoVision_Movement\\trial_0\\T1_experiment_Video_P1_NoVision_Movement_0_StartParticipantSinging_NoVision_Movement_0_EndParticipantSinging_clipped_cam1.avi', './video_split/P1\\NoVision_Movement\\trial_0\\T1_experiment_Video_P1_NoVision_Movement_0_StartParticipantSinging_NoVision_Movement_0_EndParticipantSinging_clipped_cam2.avi', './video_split/P1\\NoVision_Movement\\trial_0\\T1_experiment_Video_P1_NoVision_Movement_0_StartParticipantSinging_NoVision_Movement_0_EndParticipantSinging_clipped_cam3.avi']


Splitting the videos:   2%|▎         | 1/40 [00:12<07:58, 12.27s/file]

Participant: P1
Condition: NoVision_Movement
Trial: trial_1
saving Split Videos as: ['./video_split/P1\\NoVision_Movement\\trial_1\\T1_experiment_Video_P1_NoVision_Movement_1_StartParticipantSinging_NoVision_Movement_1_EndParticipantSinging_clipped_cam1.avi', './video_split/P1\\NoVision_Movement\\trial_1\\T1_experiment_Video_P1_NoVision_Movement_1_StartParticipantSinging_NoVision_Movement_1_EndParticipantSinging_clipped_cam2.avi', './video_split/P1\\NoVision_Movement\\trial_1\\T1_experiment_Video_P1_NoVision_Movement_1_StartParticipantSinging_NoVision_Movement_1_EndParticipantSinging_clipped_cam3.avi']


Splitting the videos:   5%|▌         | 2/40 [00:24<07:51, 12.42s/file]

...

Participant: P2
Condition: Vision_NoMovement
Trial: trial_3
saving Split Videos as: ['./video_split/P2\\Vision_NoMovement\\trial_3\\T1_experiment_Video_P2_Vision_NoMovement_3_StartParticipantSinging_Vision_NoMovement_3_EndParticipantSinging_clipped_cam1.avi', './video_split/P2\\Vision_NoMovement\\trial_3\\T1_experiment_Video_P2_Vision_NoMovement_3_StartParticipantSinging_Vision_NoMovement_3_EndParticipantSinging_clipped_cam2.avi', './video_split/P2\\Vision_NoMovement\\trial_3\\T1_experiment_Video_P2_Vision_NoMovement_3_StartParticipantSinging_Vision_NoMovement_3_EndParticipantSinging_clipped_cam3.avi']


Splitting the videos:  98%|█████████▊| 39/40 [08:14<00:13, 13.11s/file]

Participant: P2
Condition: Vision_NoMovement
Trial: trial_4
saving Split Videos as: ['./video_split/P2\\Vision_NoMovement\\trial_4\\T1_experiment_Video_P2_Vision_NoMovement_4_StartParticipantSinging_Vision_NoMovement_4_EndParticipantSinging_clipped_cam1.avi', './video_split/P2\\Vision_NoMovement\\trial_4\\T1_experiment_Video_P2_Vision_NoMovement_4_StartParticipantSinging_Vision_NoMovement_4_EndParticipantSinging_clipped_cam2.avi', './video_split/P2\\Vision_NoMovement\\trial_4\\T1_experiment_Video_P2_Vision_NoMovement_4_StartParticipantSinging_Vision_NoMovement_4_EndParticipantSinging_clipped_cam3.avi']


Splitting the videos: 100%|██████████| 40/40 [08:27<00:00, 12.68s/file]

Done, you can now look into the folder: ./video_split/


## 4. MP4 Conversion (if necessary) 
Note that you can also use online tools to convert videos like https://cloudconvert.com/

In [7]:
input_folder  = './videos_split/' 
output_folder = r "C:\Users\ahmar\OneDrive\Documents\GitHub\Mobile-Multimodal-Lab\2_PREPROCESSING\2_MOTION_TRACKING\3_freemocap\"

#Should we add this step? 
# Yes, should write some easy code with moviepy that converts whatever format into MP4. 

SyntaxError: unterminated string literal (detected at line 2) (1093415785.py, line 2)